In [1]:
import pickle
import os.path
import os 
import re
import pandas as pd
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import time
import connection
from datetime import date

def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

def get_sheet_name(SCOPES,SPREADSHEET_ID):
    title = []
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheets = sheet_metadata.get('sheets', '')
    number_of_sheets = len(sheets)
    for i in range(1, number_of_sheets):
        title.append(sheets[i].get("properties", {}).get("title"))
    return title

def clean_sheet_name(sheets_name,exclude_list):
    clean_sheets_name = filter(lambda x:x not in exclude_list, sheets_name)
    return clean_sheets_name

def ft_field_transformation(field_name):
    field_name = re.sub("(_{2})", '_', field_name)
    # matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', field_name)
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|'         # Small letter following by a capital letter
                           # '(?<=[A-Z])(?=[A-Z][a-z])|'       # Capital letter following by 1-Capital letter and then 2- Small letter Note: at this time fivetran does not support this   
                          '(?<=[0-9])(?=([a-z])|([A-Z]))|'     # Digit following by a letter either capital or small 
                          '(?<=([a-z])|([A-Z]))(?=[0-9])|'     # A letter following by digit 
                          '$)', field_name)
    string_in_list = [m.group(0) for m in matches]
    return '_'.join(string_in_list).lower()

def bi_fsm_join(bi_df,fsm_df):
    fsm_df.rename(columns = {"Field Name": "fsm_field_name", "Table Name": "fsm_table_name"},inplace = True)
    fsm_df['bi_field_name'] = fsm_df['fsm_field_name'].apply(ft_field_transformation) 
    fsm_df['bi_table_name'] = fsm_df['fsm_table_name'].apply(ft_field_transformation) 
    bi_df.rename(columns = {'Type':'bi_dataflow_type'},inplace = True)
    merge_dfs = pd.merge(bi_df,fsm_df,left_on=['Field Name','Table Name'], right_on = ['bi_field_name','bi_table_name'],how = 'left')
    return merge_dfs 

def load_dilloit_data_dictionary():
    attempt = 1
    while attempt < 10:
        try:
            print('Create deloitte_data_dictionary dataframe:')
            print('')
            scopes = ['https://www.googleapis.com/auth/spreadsheets']
            spreadsheet_id = '1649vVV3WWpdwteB1OdvxzdL8_g8Qnswj84UTT2UiJss'
            exclude_list = ['Pricebook_Entry_Priority__c','Sheet89','GuideLines','Object Summary','Object Dependency','DM Loading order','OpportunityLineItem','Franchise_QB_Data__c']
            column_names = ['Table Name','Field Name','FSL Destination Object(s)','FSL Destination Field','FSL New?','Ignore?','Migrated?']
            sheets_name = get_sheet_name(scopes,spreadsheet_id)
            clean_sheets_name = clean_sheet_name(sheets_name,exclude_list)
            deloitte_data_dictionary = pd.DataFrame(columns = column_names)
            sheet = ''
            for sheet in clean_sheets_name:
                print(sheet)
                data = pull_sheet_data(scopes,spreadsheet_id,sheet)
                for i in range(1,len(data)):
                    if len(data[i]) != len(data[0]):
                        dif = len(data[0]) - len(data[i])
                        for x in range(1,dif+1):
                            data[i].append('')
                df_data = pd.DataFrame(data[1:], columns=data[0])
                if sheet.find('(') != -1:
                    sheet = sheet[0:sheet.find('(')]

                df_data['Table Name'] = sheet
                deloitte_data_dictionary = deloitte_data_dictionary.append(df_data[column_names])
            attempt = 100
            print('deloitte data dictionary downloaded successfully')
            error_flag_1 = False
        except:
            attempt = attempt + 1  
            print('Warning: attemp number: ', attempt)
            print('Wait for 30 seconds')
            print('')
            time.sleep(30)
    if attempt == 10:
        print('cannot create deloitte_data_dictionary')
        error_flag_1 = True
    return error_flag_1, deloitte_data_dictionary

def load_bi_data_dictionary():
    attempt  =  1
    while attempt < 10:
        print('Create bi_data_dictionary dataframe:')
        try:
            print('')
            print('attempt number ',attempt )
            SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
            SPREADSHEET_ID = '1MtcIBSQuiVIPTbYTGhRm1ZHQFwFux71a5yHe5NDXDtU'
            sheet = 'Data Analysis'
            data = pull_sheet_data(SCOPES,SPREADSHEET_ID,sheet)
            bi_data_dictionary = pd.DataFrame(data[1:], columns=data[0])
            attempt = 100
            print('BI data dictionary downloaded successfully')
            error_flag_2 = False
        except:
            print('Warning! attempt number: ',attempt, ' was not successful.')
            print('Next attemp will be run in 15 seconds.')
            attempt = attempt + 1
            time.sleep(15)
    if attempt == 10:
        print('cannot create bi_data_dictionary')
        error_flag_2 = True
    return error_flag_2,bi_data_dictionary

def main_data_dictionary():
    error_flag_1,deloitte_data_dictionary = load_dilloit_data_dictionary()
    error_flag_2,bi_data_dictionary = load_bi_data_dictionary()
    if (error_flag_1 == False and error_flag_2 == False):
        joined = bi_fsm_join(bi_data_dictionary,deloitte_data_dictionary)
        joined['joined_date'] = date.today().strftime("%Y-%m-%d")
        connection.df_to_s3(joined, 'bi_fsm_data_dictionary/'+ date.today().strftime("%Y-%m-%d")+ '_fsm_bi_data_dictionary')
        connection.s3_to_redshift('bi_fsm_data_dictionary/'+ date.today().strftime("%Y-%m-%d")+ '_fsm_bi_data_dictionary','engineering_sandbox.bi_dilloit_data_dictionary')
        return 0
    else:
        return 1

main_data_dictionary()